In [29]:
import numpy as np
from glob import glob
from PIL import Image
import torchvision.transforms as T
import os
import torch
from torchvision import datasets as ds
from torch.utils.data import Dataset as D

import torchvision.models as models
import torch.nn as nn

import torch.optim as optim

In [30]:
# Mounting Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Reading Data

img_files = np.array(glob("/content/drive/MyDrive/CSGO/data/Train/*/*"))
print('%d train images' % len(img_files))

img_files = np.array(glob("/content/drive/MyDrive/CSGO/data/Test/*/*"))
print('%d test images' % len(img_files))

img_files = np.array(glob("/content/drive/MyDrive/CSGO/data/*/*/*"))
print('%d total images' % len(img_files))

827 train images
66 test images
893 total images


In [32]:
# Apply a list of sequential operations on train and test datasets

# T.Resize(320) is resize the input image to this height

# T.CenterCrop(256) is to crop the center pixels e.g. if image is 288*288 and the crop is 256*256 then 16 pixels are cropped 
# from top, left, bottom and right 

# T.ToTensor() is to convert image into pytorch tensor

# T.Normalize() to make the data distribution similar and hence convergence is faster, 
# Subtract the mean and standard deviation (known from ImageNet) from our dataset

train_transform = T.Compose([T.Resize(320),
                            T.CenterCrop(256),
                            T.ToTensor(),
                            T.Normalize(
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]
                            )
                  ])

test_transform = T.Compose([T.Resize(size=(256, 256)),
                            T.ToTensor(),
                            T.Normalize(
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]
                            )
                  ])

train_data = ds.ImageFolder('/content/drive/MyDrive/CSGO/data/Train', transform=train_transform)
test_data = ds.ImageFolder('/content/drive/MyDrive/CSGO/data/Test', transform=test_transform)

In [33]:
# The batch_size is set to 32 means that when you iterate through the Dataset, 
# DataLoader will output 32 instances of data instead of one

# shuffle will reshuffle the data at each epoch, this prevents the model from learning
# the order of training data

# num_workers is how many subprocesses to use for data loading.

# drop_last is to drop the last batch if the dataset size is not divisible by the batch size

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=32,
                                           shuffle=True,
                                           num_workers=2,
                                           drop_last=False)
test_loader = torch.utils.data.DataLoader(test_data,
                                           batch_size=32,
                                           shuffle=False,
                                           num_workers=2,
                                           drop_last=False)

loaders_scratch = {
    'train': train_loader,
    'test': test_loader
}

In [34]:
# Transfer Learning : Using large networks that were trained with vast datasets for our new tasks reduces
# time and computation requirements. With transfer learning, we can build good classifiers with a few hundred images

loaders_transfer = loaders_scratch.copy()

model_transfer = models.resnet50(pretrained=True)

# Freezing weights

for param in model_transfer.parameters():
    param.requires_grad = False

# Last fully connected layer

model_transfer.fc = nn.Linear(2048, 33)

for param in model_transfer.fc.parameters():
    param.requires_grad = True

if use_cuda:
    model_transfer = model_transfer.cuda()

# Defining loss function with learning rate

criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.Adam(model_transfer.fc.parameters(), lr=0.001)

# Training function

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda):
    """returns trained model"""
    
    for epoch in range(1, n_epochs+1):
        # Variable initialization
        train_loss = 0.0
        
        # MODEL TRAINING #
        
        model_transfer.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            # find training loss and update model parameters

            optimizer.zero_grad()
            
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
  
        # print training error for each epoch 
        print('Epoch: {} \tTraining Error: {:.6f}'.format(
            epoch, 
            train_loss))
   
    # return trained model
    return model

# Call Trainiing

train(10, loaders_transfer, model_transfer, optimizer_transfer,
      criterion_transfer, use_cuda)

Epoch: 1 	Training Loss: 3.045330
Epoch: 2 	Training Loss: 1.685128
Epoch: 3 	Training Loss: 0.976012
Epoch: 4 	Training Loss: 0.607852
Epoch: 5 	Training Loss: 0.419451
Epoch: 6 	Training Loss: 0.311695
Epoch: 7 	Training Loss: 0.254488
Epoch: 8 	Training Loss: 0.201369
Epoch: 9 	Training Loss: 0.175340
Epoch: 10 	Training Loss: 0.135175


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [35]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

Test Loss: 0.628141


Test Accuracy: 87% (58/66)
